# https://github.com/ramalho/pyob

<img src="img/title-card.png" width="720" title="ThoughtWorks presents Pythonic Objects by Luciano Ramalho">

# Attribute basics

## Smalltalk class declaration

<img src="img/finhist-browser.png" width="720" title="Smallalk-80 class browser">

**Figure 17.17 from Smalltalk-80, the language**
<br>Class browser showing definition of a ``FinancialHistory`` class with three instance variables: ``cashOnHand``, ``incomes``, and ``expenditures``.

## Official Java tutorial

The next two figures are from the **Java Tutorial (Sun/Oracle)**, section [What is an object?](https://docs.oracle.com/javase/tutorial/java/concepts/object.html).

An object is depicted as fields surrounded by methods:

<img src="img/concepts-object.gif" title="An object">

Quoting from [What is an object?](https://docs.oracle.com/javase/tutorial/java/concepts/object.html):

> Methods operate on an object's internal state and serve as the primary mechanism for object-to-object communication. Hiding internal state and requiring all interaction to be performed through an object's methods is known as *data encapsulation* — a fundamental principle of object-oriented programming.

An object representing a bicyle has methods such as *Change gear* and *Brake*, and fields such as *speed* and *cadence*:

<img src="img/concepts-bicycleObject.gif" title="A bicycle object">

Code from section [What is a class?](https://docs.oracle.com/javase/tutorial/java/concepts/class.html) from the **Java Tutorial**.

```java
class Bicycle {

    int cadence = 0;
    int speed = 0;
    int gear = 1;

    void changeCadence(int newValue) {
         cadence = newValue;
    }
    
    //...
}    
```

## What about Python?

### Python terms

From the **Python tutorial**, section [9.3.3. Instance Objects](https://docs.python.org/3.7/tutorial/classes.html#instance-objects)

> There are two kinds of valid attribute names, data attributes and methods.
>
> *Data attributes* correspond to “instance variables” in Smalltalk, and to “data members” in C++.

In Python, the generic term *attribute* refers to both *fields* and *methods* in Java:

Python term    |Java concept
:----------    |:-----------
attribute      | fields and methods
data attribute | field
method         | method



### Python < 3.6

* No way to declare instance variables without assigning.
* No way to declare variables at all (except function arguments).
* First assignment is the "declaration".
* Attributes defined in a class body are *class attributes*.


### Descriptors

Descriptors are defined in a class body, so they are *class attributes*.

#### Descriptor examples

From [Django Models](https://docs.djangoproject.com/en/2.2/topics/db/models/):

```python
from django.db import models

class Musician(models.Model):
    first_name = models.CharField(max_length=50)
    last_name = models.CharField(max_length=50)
    instrument = models.CharField(max_length=100)

class Album(models.Model):
    artist = models.ForeignKey(Musician, on_delete=models.CASCADE)
    name = models.CharField(max_length=100)
    release_date = models.DateField()
    num_stars = models.IntegerField()
```

ORMs use *descriptors* to declare fields (eg. Django, SQLAlchemy) that manage the persistency of the data attributes of instances that are database records.

Such data-oriented descriptors are not part of the Python Standard Library—they are provided by external framweorks. 

## Back to basics

Check the version of Python we are using:

In [1]:
import sys
print(sys.version)

3.7.3 (default, Mar 27 2019, 16:54:48) 
[Clang 4.0.1 (tags/RELEASE_401/final)]


## A simplistic class

In [2]:
class Coordinate:
    '''Coordinate on Earth'''

In [3]:
cle = Coordinate()
cle.lat = 41.4
cle.long = -81.8
cle

In [4]:
cle.lat

41.4

### First method: ``__repr__``

In [5]:
class Coordinate:
    '''Coordinate on Earth'''
        
    def __repr__(self):
        return f'Coordinate({self.lat}, {self.long})'   

In [6]:
cle = Coordinate()
cle.lat = 41.4
cle.long = -81.8
cle

Coordinate(41.4, -81.8)

In [8]:
cle.__repr__()

'Coordinate(41.4, -81.8)'

In [9]:
repr(cle)

'Coordinate(41.4, -81.8)'

### About ``__repr__``

* Good for exploratory programming, documentation, doctests, and debugging.
* Best practice: if viable, make ``__repr__`` return string with syntax required to create a new instance like the one inspected (i.e. ``eval(repr(x)) == x``)
* If not viable, use ``<MyClass ...>`` with some ``...`` that identifies the particular instance.


### ``__repr__`` v. ``__str__``

* ``__repr__`` is for programming displays.
* ``__str__`` is for end-user displays.

### ``__str__`` example

In [15]:
class Coordinate:
    '''Coordinate on Earth'''
    
    lat = 0.0
    long = 0.0
        
    def __repr__(self):
        return f'Coordinate({self.lat}, {self.long})'
    
    def __str__(self):
        ns = 'NS'[self.lat < 0]
        we = 'EW'[self.long < 0]
        return f'{abs(self.lat):.1f}°{ns}, {abs(self.long):.1f}°{we}'

In [11]:
cle = Coordinate()
cle.lat = 41.4
cle.long = -81.8
print(cle)

41.4°N, 81.8°W


### But...

In [16]:
gulf_of_guinea = Coordinate()
try:
    print(gulf_of_guinea)
except AttributeError as e:
    print(e)

0.0°N, 0.0°E


> **Quick fix**: add class attributes to provide defaults.

## Class attributes as defaults

In [17]:
class Pizza:
    
    diameter = 40  # cm
    slices = 8

    flavor = 'Cheese'
    flavor2 = None

In [18]:
p = Pizza()
p.slices

8

In [19]:
p.flavor

'Cheese'

In [20]:
p.__dict__

{}

In [22]:
p.flavor = 'Sausage'
p.__dict__

{'flavor': 'Sausage'}

In [23]:
p2 = Pizza()
p2.flavor

'Cheese'

In [24]:
Pizza.__dict__

mappingproxy({'__module__': '__main__',
              'diameter': 40,
              'slices': 8,
              'flavor': 'Cheese',
              'flavor2': None,
              '__dict__': <attribute '__dict__' of 'Pizza' objects>,
              '__weakref__': <attribute '__weakref__' of 'Pizza' objects>,
              '__doc__': None})

## A better pizza

In [25]:
class Pizza:

    diameter = 40  # cm
    slices = 8

    def __init__(self, flavor='Cheese', flavor2=None):
        self.flavor = flavor
        self.flavor2 = flavor2

Good practices shown here:

* use of *class attributes* for attributes shared by all instances;
* attributes that are expected to vary among instances are *instance attributes*;
* instance attributes are *all* assigned in ``__init__``;
* default values for instance attributes are ``__init__`` argument defaults.

[PEP 412 — Key-Sharing Dictionary](https://www.python.org/dev/peps/pep-0412/) introduced an optimization that saves memory when instances of a class have the same instance attribute names set on ``__init__``.

## Lab #1: enhancing ``Coordinate``


Follow instructions at [labs/1/README.rst](https://github.com/ramalho/pyob/blob/master/labs/01-coordinate/README.rst).

In [ ]:
import geohash

class Coordinate:
    '''Coordinate on Earth'''
    
    reference_system = 'WGS84'
    
    def __init__(self, lat, long):
        self.lat = lat
        self.long = long
    
    def __repr__(self):
        return f'Coordinate({self.lat}, {self.long})'
    
    def __str__(self):
        ns = 'NS'[self.lat < 0]
        we = 'WE'[self.long < 0]
        return f'{abs(self.lat):.1f}°{ns}, {abs(self.long):.1f}°{we}'
    
    def geohash(self):
        return geohash.encode(self.lat, self.long)
    

In [ ]:
cle = Coordinate(41.5, -81.7)
cle.geohash()

In [ ]:
from dataclasses import InitVar
from typing import ClassVar

In [ ]:
default_flavor = 'Cheese'

class Pizza:
    
    def __init__(self, flavor1=default_flavor, flavor2=None):
        self.flavor1 = flavor1
        self.flavor2 = flavor2

In [ ]:
InitVar?

----

## dataclass options

```
@dataclasses.dataclass(*, 
    init=True, repr=True, eq=True, order=False, unsafe_hash=False, frozen=False)
```

<table>
<tr><th>option</th><th>default</th><th style="text-align: left;">meaning</th></tr>
<tr><td>init</td><td>True</td>
    <td style="text-align: left;">generate <code>__init__</code>¹</td></tr>
<tr><td>repr</td><td>True</td>
    <td style="text-align: left;">generate <code>__repr__</code>¹</td></tr>
<tr><td>eq</td><td>True</td>
    <td style="text-align: left;">generate <code>__eq__</code>¹</td></tr>
<tr><td>order</td><td>False</td>
    <td style="text-align: left;">generate <code>__lt__</code>, <code>__le__</code>, <code>__gt__</code>, <code>__ge__</code>²</td></tr>
<tr><td>unsafe_hash</td><td>False</td>
    <td style="text-align: left;">generate <code>__hash__</code>³</td></tr>
<tr><td>frozen</td><td>False</td>
    <td style="text-align: left;">make instances "immutable" ⁴</td></tr>
</table>

**Notes**

¹ Ignored if the special method is implemented by user.<br>
² Raises exceptions if ``eq=False`` or any of the listed special methods are implemented by user.<br>
³ Complex semantics and several caveats — see: [dataclass documentation](https://docs.python.org/3/library/dataclasses.html#dataclasses.dataclass).<br>
⁴ Not really immutable — imutability is emulated generating ``__setattr__`` and ``__delattr__`` which raise ``dataclass.FrozenInstanceError`` (a subclass of ``AttributeError``).

In [ ]:
import dataclasses
dataclasses.FrozenInstanceError.__mro__

<img src="img/thoughtworks.png" width="300" title="ThoughtWorks, Inc. logo">
